<a href="https://colab.research.google.com/github/realalyeasin/Machine-Learning/blob/main/%F0%9F%94%A0Movie_Review_Classification_%E2%9A%99%EF%B8%8F%7C%7C_TF_Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

In [3]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [4]:
print("Version : ", tf.__version__)
print("Egar Mode : ", tf.executing_eagerly())
print("Hub : ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "not available")

Version :  2.5.0
Egar Mode :  True
Hub :  0.12.0
GPU is available


In [5]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews", split=('train[60%:]', 'train[:60%]', 'test'), as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteSQYDFR/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteSQYDFR/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [8]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [9]:
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was a superb episode, one of the best of both seasons. Down right horror for a change, with a story that is way way above the average MOH episodes, if there is such a thing. A man's wife is almost burned to death in a tragic car wreck, in which he was driving. His airbag worked, her's didn't. She is burned beyond recognition (great makeup btw), and not given much of a chance to live without a full skin graft. BUT, even in a coma, she keeps dying but brought back by modern technology, and when she does die for a few minutes, her ghost appears as a very vengeful spirit. Carnage of course ensues, and also some extremely gory killings, and also, some extremely sexy scenes. What more could you ask for, you might ask? Well, not much, because this baby has it all, and a very satirical ending, that should leave a smile on most viewers faces. I just loved Rob Schmidt's (Wrong Turn) direction too, he has a great knack for horror. Excell

In [10]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 0, 1])>

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim128/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype = tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

In [12]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               124642688 
_________________________________________________________________
dense (Dense)                (None, 16)                2064      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 124,644,769
Trainable params: 124,644,769
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer='adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
history = model.fit(train_data.shuffle(1000).batch(512), 
                    epochs = 10,
                    validation_data=validation_data.batch(512),
                    verbose = 1)

Epoch 1/10
20/20 [==============================] - 6s 155ms/step - loss: 0.6740 - accuracy: 0.5078 - val_loss: 0.6403 - val_accuracy: 0.5766
Epoch 2/10
20/20 [==============================] - 3s 148ms/step - loss: 0.5863 - accuracy: 0.6449 - val_loss: 0.5462 - val_accuracy: 0.7399
Epoch 3/10
20/20 [==============================] - 3s 149ms/step - loss: 0.4488 - accuracy: 0.8116 - val_loss: 0.4282 - val_accuracy: 0.8125
Epoch 4/10
20/20 [==============================] - 3s 149ms/step - loss: 0.3051 - accuracy: 0.8928 - val_loss: 0.3565 - val_accuracy: 0.8543
Epoch 5/10
20/20 [==============================] - 3s 152ms/step - loss: 0.2021 - accuracy: 0.9334 - val_loss: 0.3264 - val_accuracy: 0.8667
Epoch 6/10
20/20 [==============================] - 3s 156ms/step - loss: 0.1306 - accuracy: 0.9636 - val_loss: 0.3086 - val_accuracy: 0.8656
Epoch 7/10
20/20 [==============================] - 3s 158ms/step - loss: 0.0849 - accuracy: 0.9818 - val_loss: 0.3109 - val_accuracy: 0.8653
Epoch 

In [17]:
result = model.evaluate(test_data.batch(512), verbose=2)

49/49 - 2s - loss: 0.3639 - accuracy: 0.8546


In [18]:
for name, value in zip(model.metrics_names, result):
  print("%s : %3.f"%(name, value))

loss :   0
accuracy :   1
